## Import library

In [1]:
# importing required libraries
import os, sys
import sys
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib')
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')
import loader


import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math 
import timeit
import evaluation as evl

from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Inputs
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Data viz
from mlens.visualization import corr_X_y, corrmat

# Model evaluation
from mlens.metrics import make_scorer
from mlens.model_selection import Evaluator

# Ensemble
from mlens.ensemble import SuperLearner


/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


## Load data

In [2]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-1000.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-1000.test.npz"

In [3]:
def load_data(path, i_exp):
    col =  ["t", "yf", "ycf", "mu0", "mu1" ]
    cov = ["x" + str(i) for i in range(1,26)]
    col = col + cov
    features = cov + ["t"] 
    D = np.load(path)
    df = pd.DataFrame(columns=col)

    for i in range(1,26):
        df['x' + str(i)]  = D['x'][:,i-1,i_exp-1]

    df['t']  = D['t'][:,i_exp-1:i_exp]
    df['yf'] = D['yf'][:,i_exp-1:i_exp]
    df['ycf'] = D['ycf'][:,i_exp-1:i_exp]
    df['mu0'] = D['mu0'][:,i_exp-1:i_exp]
    df['mu1'] = D['mu1'][:,i_exp-1:i_exp]
    return df

In [4]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"] 

## Influence function

In [174]:
data_train = load_data(PATH_TRAIN, 40)
data_test  = load_data(PATH_TEST, 40)

In [175]:
data_train.head()

,t,yf,ycf,mu0,mu1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,0.0,4.881007,10.152449,4.124143,11.114897,1.159618,-0.202946,-0.733261,-0.879606,0.808706,-0.360940,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,4.191785,11.010013,4.631141,11.230842,-0.504825,-0.202946,0.011465,2.244320,0.558638,-0.692171,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,11.657644,9.277814,8.799111,11.872689,-0.137351,0.196818,0.011465,2.244320,1.746464,0.135907,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,18.054683,12.153734,18.181274,12.598431,0.208507,0.996346,1.128554,0.161703,-0.504155,1.129600,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,6.729148,10.984696,4.917858,11.290912,-1.045229,-1.337276,1.128554,-0.879606,0.683672,0.301522,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [176]:
con_feature = ["x" + str(a) for a in range(1,7)]
cat_feature = ["x" + str(a) for a in range(7, 26)]

In [177]:
# scaler = StandardScaler()
# data_test[con_feature] = scaler.fit_transform(data_test[con_feature])
# data_train[con_feature] = scaler.fit_transform(data_train[con_feature])

In [178]:
delta = 2.5
features = cov.copy()
features.append(treatment)

## Fit treatment
model_t = RandomForestClassifier(n_estimators=10)
model_t.fit(data_train[cov], data_train[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data_train[features], data_train[outcome])

## Compute propensity score
data_test['p1'] = model_t.predict_proba(data_test[cov])[:,1]
data_test['p0'] = 1 - data_test['p1']

## fit outcome
data_test['predicted_y'] = model_y.predict(data_test[features])

## Compute counterfactual outcome with no treatment
data_pos = data_test.copy()
data_pos[treatment] = 1
data_test['cf1'] = model_y.predict(data_pos[features])

## Compute counterfactual outcome with treatment
data_neg = data_test.copy()
data_neg[treatment] = 0
data_test['cf0'] = model_y.predict(data_neg[features])


In [179]:
# ## Fit treatment
# model_t = LogisticRegression()
# model_t.fit(data_train[cov], data_train[treatment])

# data_train['p1'] = model_t.predict_proba(data_train[cov])[:,0]
# data_train['p0'] = 1 - data_train['p1']

# clf = RandomForestClassifier(n_estimators=10)

# #Train the model using the training sets y_pred=clf.predict(X_test)
# clf.fit(data_train[cov], data_train[treatment])

# y_pred=clf.predict(data_train[cov])

# data_train['p1'] = clf.predict_proba(data_train[cov])[:,1]
# data_train['p0'] = 1 - data_train['p1']


In [180]:
data_train.t.value_counts()

0.0    546
1.0    126
Name: t, dtype: int64

In [182]:
model_t.predict(data_train[cov])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [149]:
print("Accuracy:",metrics.accuracy_score(model_t.predict(data_test[cov]), data_test[treatment]))

Accuracy: 0.7733333333333333


**Incremental propensity score:**
$$q_1 = \frac{\delta p_0}{\delta p_0 + p_1}$$


In [150]:
## Compute incremental score 
data_test['q1'] = (delta * data_test['p1']) / (delta * data_test['p1'] + data_test['p0'])
data_test['q0'] = 1 - data_test['q1']

**Estimator:**
$$\Psi (\delta ) = q1*\mathbb{E}[Y | X,A=1] + q_0*\mathbb{E}[Y | X,A=0]$$

In [151]:
simple_estimator = data_test['q1']*data_test['cf1'] + data_test['q0']*data_test['cf0']

In [152]:
simple_estimator_2 = (delta*data_test[treatment] + 1 - data_test[treatment])*data_test[outcome] / (
    delta*data_test['p1'] + data_test['p0']
)

In [153]:
num_units = data_test.shape[0]
data_test['ips_weight'] = (1/num_units) * (
    data_test[treatment] / data_test['p1'] + (1 - data_test[treatment]) / (1 - data_test['p1'])
)

# Calculating the effect
data_test['d_y'] = (
    data_test['ips_weight'] *
    data_test[treatment] *
    data_test[outcome]
)
data_test['dbar_y'] = (
    data_test['ips_weight'] *
    (1 - data_test[treatment]) *
    data_test[outcome]
)
weighting_estimator  = data_test['d_y'] - data_test['dbar_y']


**Influence function** $\mathbb{1}$

$$ q_0*\phi_0 + q_1*\phi_1 + \frac{\delta*(\mu(X,1) - \mu(X,0)*(A - p_1)}{(\delta*p_1 + p_0)^2} - \Psi (\delta )$$ 

$$\phi_a = \frac{\mathbb{1}(A=a)}{\pi(a|x)}(Y - \mu(X,A)) +  \mu(X,a)$$

In [154]:
data_test['predicted_t'] = model_t.predict(data_test[cov])

In [155]:
# Y - mu(X,A)
residual = data_test[outcome] - data_test['predicted_y']

w0 = [0 if data_test.loc[i,'predicted_t'] == 1 else 1/data_test.loc[i, 'p0'] for i in range(len(data_test))]
w1 = [0 if data_test.loc[i,'predicted_t'] == 0 else 1/data_test.loc[i, 'p1'] for i in range(len(data_test))]
w0 = np.array(w0)
w1 = np.array(w1)

phi_0 = w0 * residual + data_test['cf0'] 
phi_1 = w1 * residual + data_test['cf1']

avg_phi = data_test['q1']*phi_1 + data_test['q0']*phi_0

diff_cf = data_test['cf1'] - data_test['cf0']
diff_ps = data_test[treatment] - data_test['p1']
modi_ps = (delta*data_test['p1'] + data_test['p0'])**2

avg_diff = (delta*diff_cf*diff_ps) / modi_ps

In [156]:
# data_test['w0'] = [0 if data_test.loc[i,treatment] == 1 else 1/data_test.loc[i, 'p0'] for i in range(len(data_test))]
# data_test['w1'] = [0 if data_test.loc[i,treatment] == 0 else 1/data_test.loc[i, 'p1'] for i in range(len(data_test))]

In [157]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)


In [158]:
# data_test[['p0', 'p1', 'w0', 'w1', 't']]

In [159]:
# w0, w1

In [160]:
# data_test[['t', 'p0', 'p1']]

In [162]:
true_effect = data_test['mu1'] - data_test['mu0']
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

True effect: 3.511+-0.594


In [163]:
influence = avg_phi + avg_diff - simple_estimator

In [164]:
means, stds = np.mean(influence, axis=0), sem(influence, axis=0)
print('Estimation effect: {:.3f}+-{:.3f}'.format(means, stds))

Estimation effect: 0.307+-0.400


In [142]:
data_test[['yf', 'cf0', 'cf1', 'predicted_y', 't', 'p0', 'p1', 'predicted_t']]

,yf,cf0,cf1,predicted_y,t,p0,p1,predicted_t
0,2.693778,2.768257,10.287870,2.768257,0.0,1.0,0.0,0.0
1,12.388813,6.789495,10.043943,10.043943,1.0,0.7,0.3,0.0
2,5.559540,5.348458,10.218338,5.348458,0.0,0.9,0.1,0.0
3,15.413086,13.204406,14.240686,13.204406,0.0,0.8,0.2,0.0
4,9.180642,8.503391,12.769866,8.503391,0.0,0.9,0.1,0.0
5,8.526179,9.957633,13.718344,9.957633,0.0,0.9,0.1,0.0
6,3.417452,2.922498,10.815050,2.922498,0.0,1.0,0.0,0.0
7,7.485668,8.318328,12.575033,8.318328,0.0,0.8,0.2,0.0
8,3.837117,3.068015,9.587836,3.068015,0.0,0.9,0.1,0.0
9,9.663785,8.178124,13.315039,8.178124,0.0,0.9,0.1,0.0


In [140]:
# data_test[data_test.t == 1]

In [19]:
---

SyntaxError: invalid syntax (<ipython-input-19-29e0c3615294>, line 1)

## Sample method

In [183]:
def cal_influence_function(delta, data_train, data_test):
    features = cov.copy()
    features.append(treatment)

    ## Fit treatment
    model_t = RandomForestClassifier(n_estimators=10)
    model_t.fit(data_train[cov], data_train[treatment])

    ## Fit outcome
    ## model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    ## model_y.fit(data_train[features], data_train[outcome])
    
    data_train_treated = data_train[data_train[treatment] == 1]
    data_train_control = data_train[data_train[treatment] == 0]
    
    model_y_treated = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y_treated.fit(data_train_treated[cov], data_train_treated[outcome])
 
    model_y_control = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y_control.fit(data_train_control[cov], data_train_control[outcome])
    
    
    ## Compute propensity score
    data_test['p1'] = model_t.predict_proba(data_test[cov])[:,1]
    data_test['p0'] = 1 - data_test['p1']
    
    data_test['predicted_y'] = 0

    data_test_treated = data_test[data_test[treatment] == 1]
    data_test_control = data_test[data_test[treatment] == 0]
    
    ## fit outcome
    data_test.loc[data_test_treated.index.values,'predicted_y'] = model_y_treated.predict(data_test_treated[cov])
    data_test.loc[data_test_control.index.values,'predicted_y'] = model_y_control.predict(data_test_control[cov])

    ## Compute counterfactual outcome with no treatment
    data_pos = data_test.copy()
    data_pos[treatment] = 1
    data_test['cf1'] = model_y_treated.predict(data_pos[cov])

    ## Compute counterfactual outcome with treatment
    data_neg = data_test.copy()
    data_neg[treatment] = 0
    data_test['cf0'] = model_y_control.predict(data_neg[cov])
    
    ## Compute incremental score 
    data_test['q1'] = (delta * data_test['p1']) / (delta * data_test['p1'] + data_test['p0'])
    data_test['q0'] = 1 - data_test['q1']
    estimator = data_test['q1']*data_test['cf1'] + data_test['q0']*data_test['cf0']

    # Y - mu(X,A)
    residual = data_test[outcome] - data_test['predicted_y']
    data_test['predicted_t'] = model_t.predict(data_test[cov])
    
    print(data_test[['t', 'p0', 'p1']])
    
    w0 = [0 if data_test.loc[i,'predicted_t'] == 1 else 1/data_test.loc[i, 'p0'] for i in range(len(data_test))]
    w1 = [0 if data_test.loc[i,'predicted_t'] == 0 else 1/data_test.loc[i, 'p1'] for i in range(len(data_test))]
    w0 = np.array(w0)
    w1 = np.array(w1)

    phi_0 = w0 * residual + data_test['cf0'] 
    phi_1 = w1 * residual + data_test['cf1']

    avg_phi = data_test['q1']*phi_1 + data_test['q0']*phi_0

    diff_cf = data_test['cf1'] - data_test['cf0']
    diff_ps = data_test[treatment] - data_test['p1']
    modi_ps = (delta*data_test['p1'] + data_test['p0'])**2

    avg_diff = (delta*diff_cf*diff_ps) / modi_ps
    influence = avg_phi + avg_diff - estimator
    return influence


In [184]:
true_effect = np.mean(data_test['mu1'] - data_test['mu0'])

In [186]:
k = 2
num_of_delta = 1
delta_seq = np.linspace(0.5, 5, num_of_delta)

kf = KFold(n_splits = k, shuffle = True, random_state = 1)
effect = []
fold = 0

influence_sample = []
for train_index, test_index in tqdm(kf.split(data_train)):
    
    df_train = data_train.iloc[train_index]
    df_test = data_train.iloc[test_index]
    
    df_train = df_train.reset_index().drop(columns = ['index'])
    df_test = df_test.reset_index().drop(columns = ['index'])

    
    influence_delta = []
    
    for sample_delta in tqdm(delta_seq):
        influence = cal_influence_function(sample_delta, df_train, data_test)
        influence_delta.append(np.mean(influence))
        
    influence_sample.append(np.mean(influence_delta))

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:02<00:00,  2.40s/it]
1it [00:02,  2.41s/it]
  0%|          | 0/1 [00:00<?, ?it/s]

      t   p0   p1
0   0.0  1.0  0.0
1   1.0  0.7  0.3
2   0.0  0.7  0.3
3   0.0  0.8  0.2
4   0.0  1.0  0.0
5   0.0  0.4  0.6
6   0.0  1.0  0.0
7   0.0  0.8  0.2
8   0.0  0.9  0.1
9   0.0  0.9  0.1
10  1.0  1.0  0.0
11  0.0  0.8  0.2
12  0.0  1.0  0.0
13  0.0  0.4  0.6
14  1.0  0.8  0.2
15  1.0  0.7  0.3
16  0.0  1.0  0.0
17  0.0  0.6  0.4
18  0.0  0.9  0.1
19  0.0  1.0  0.0
20  1.0  0.6  0.4
21  0.0  0.8  0.2
22  0.0  0.8  0.2
23  0.0  1.0  0.0
24  0.0  0.9  0.1
25  1.0  1.0  0.0
26  0.0  0.6  0.4
27  0.0  0.8  0.2
28  0.0  1.0  0.0
29  0.0  1.0  0.0
30  0.0  0.6  0.4
31  1.0  0.7  0.3
32  0.0  0.8  0.2
33  0.0  0.7  0.3
34  0.0  0.9  0.1
35  0.0  0.4  0.6
36  0.0  0.5  0.5
37  0.0  0.9  0.1
38  0.0  0.9  0.1
39  0.0  1.0  0.0
40  0.0  0.8  0.2
41  0.0  1.0  0.0
42  0.0  0.8  0.2
43  0.0  1.0  0.0
44  0.0  0.4  0.6
45  0.0  0.7  0.3
46  0.0  0.9  0.1
47  0.0  0.7  0.3
48  1.0  1.0  0.0
49  0.0  0.8  0.2
50  0.0  0.7  0.3
51  0.0  0.8  0.2
52  1.0  0.8  0.2
53  0.0  0.7  0.3
54  0.0  0


100%|██████████| 1/1 [00:02<00:00,  2.48s/it]
2it [00:04,  2.45s/it]

      t   p0   p1
0   0.0  1.0  0.0
1   1.0  0.8  0.2
2   0.0  0.7  0.3
3   0.0  0.8  0.2
4   0.0  0.8  0.2
5   0.0  0.5  0.5
6   0.0  0.9  0.1
7   0.0  1.0  0.0
8   0.0  0.6  0.4
9   0.0  0.9  0.1
10  1.0  0.9  0.1
11  0.0  1.0  0.0
12  0.0  0.9  0.1
13  0.0  1.0  0.0
14  1.0  0.8  0.2
15  1.0  0.9  0.1
16  0.0  0.6  0.4
17  0.0  1.0  0.0
18  0.0  0.8  0.2
19  0.0  1.0  0.0
20  1.0  0.9  0.1
21  0.0  1.0  0.0
22  0.0  0.9  0.1
23  0.0  0.8  0.2
24  0.0  0.9  0.1
25  1.0  0.6  0.4
26  0.0  0.6  0.4
27  0.0  0.8  0.2
28  0.0  1.0  0.0
29  0.0  0.9  0.1
30  0.0  0.4  0.6
31  1.0  0.6  0.4
32  0.0  0.6  0.4
33  0.0  0.6  0.4
34  0.0  0.8  0.2
35  0.0  0.8  0.2
36  0.0  0.8  0.2
37  0.0  0.9  0.1
38  0.0  1.0  0.0
39  0.0  1.0  0.0
40  0.0  0.5  0.5
41  0.0  1.0  0.0
42  0.0  0.5  0.5
43  0.0  0.4  0.6
44  0.0  0.5  0.5
45  0.0  0.5  0.5
46  0.0  0.6  0.4
47  0.0  0.7  0.3
48  1.0  1.0  0.0
49  0.0  0.9  0.1
50  0.0  0.6  0.4
51  0.0  0.8  0.2
52  1.0  0.8  0.2
53  0.0  0.6  0.4
54  0.0  0

In [171]:
np.mean(influence_sample)

In [172]:
true_effect

## Calculate with multiple delta in data_test

In [ ]:
influence_delta = []
delta_seq = np.linspace(1.5, 10, 20)
 
for sample_delta in tqdm(delta_seq):
    influence = cal_influence_function(sample_delta, data_train, data_test)
    influence_delta.append(np.mean(influence))

In [ ]:
np.mean(influence_delta)

In [ ]:
true_effect = data_test['mu1'] - data_test['mu0']

In [ ]:
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

## Super Learners

In [187]:
def cal_influence_function_model(delta, data_train, data_test, model_y, model_t, cov, treatment, outcome):
    features = cov.copy()
    features.append(treatment)

    ## Compute propensity score
    data_test['p1'] = model_t.predict_proba(data_test[cov])[:,1]
    data_test['p0'] = 1 - data_test['p1']
    
    ## fit outcome
    
    data_test['predicted_y'] = model_y.predict(data_test[features])

    ## Compute counterfactual outcome with no treatment
    data_pos = data_test.copy()
    data_pos[treatment] = 1
    data_test['treated_cf_outcome'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data_test.copy()
    data_neg[treatment] = 0
    data_test['control_cf_outcome'] = model_y.predict(data_neg[features])
    
    ## Compute incremental score 
    data_test['q1'] = (delta * data_test['p1']) / (delta * data_test['p1'] + data_test['p0'])
    data_test['q0'] = 1 - data_test['q1']
    estimator = data_test['q1']*data_test['treated_cf_outcome'] + data_test['q0']*data_test['control_cf_outcome']

    # Y - mu(X,A)
    residual = data_test[outcome] - data_test['predicted_y']

    w0 = [0 if data_test.loc[i,treatment] == 1 else 1/data_test.loc[i, 'p0'] for i in range(len(data_test))]
    w1 = [0 if data_test.loc[i,treatment] == 0 else 1/data_test.loc[i, 'p1'] for i in range(len(data_test))]
    w0 = np.array(w0)
    w1 = np.array(w1)

    phi_0 = w0 * residual + data_test['treated_cf_outcome'] 
    phi_1 = w1 * residual + data_test['control_cf_outcome']

    avg_phi = data_test['q1']*phi_1 + data_test['q0']*phi_0

    diff_cf = data_test['treated_cf_outcome'] - data_test['control_cf_outcome']
    diff_ps = data_test[treatment] - data_test['p1']
    modi_ps = (delta*data_test['p1'] + data_test['p0'])**2

    avg_diff = (delta*diff_cf*diff_ps) / modi_ps
    influence = avg_phi + avg_diff - estimator
    return influence

In [188]:
# example of a super learner for regression using the mlens library
from math import sqrt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from mlens.ensemble import SuperLearner
 
# create a list of base-models
def get_models():
    models = list()
    models.append(LinearRegression())
    models.append(ElasticNet())
    models.append(SVR(gamma='scale'))
    models.append(DecisionTreeRegressor())
    models.append(KNeighborsRegressor())
    models.append(AdaBoostRegressor())
    models.append(BaggingRegressor(n_estimators=10))
    models.append(RandomForestRegressor(n_estimators=10))
    models.append(ExtraTreesRegressor(n_estimators=10))
    return models
 
# cost function for base models
def rmse(yreal, yhat):
    return sqrt(mean_squared_error(yreal, yhat))
 
# create the super learner
def get_super_learner(X):
    ensemble = SuperLearner(scorer=rmse, folds=10, shuffle=True, sample_size=len(X))
    # add base models
    models = get_models()
    ensemble.add(models)
    # add the meta model
    ensemble.add_meta(LinearRegression())
    return ensemble


In [189]:
data_train = load_data(PATH_TRAIN, 40)
data_test  = load_data(PATH_TEST, 40)

In [ ]:
k = 5
num_of_delta = 10
delta_seq = np.linspace(0.5, 5, num_of_delta)

kf = KFold(n_splits = k, shuffle = True, random_state = 1)

influence_sample = []
for train_index, test_index in tqdm(kf.split(data_train)):
    
    df_train = data_train.iloc[train_index]
    df_test = data_train.iloc[test_index]
    
    df_train = df_train.reset_index().drop(columns = ['index'])
    df_test = df_test.reset_index().drop(columns = ['index'])

    model_y = get_super_learner(df_train)
    model_y.fit(df_train[features].values, df_train[outcome].values)
    
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(df_train[cov], df_train[treatment])
    
    influence_delta = []
    
    for sample_delta in tqdm(delta_seq):
        influence = cal_influence_function_model(sample_delta, df_train, df_test, model_y, model_t, cov, treatment, outcome)
        influence_delta.append(np.mean(influence))
        
    influence_sample.append(np.mean(influence_delta))

In [ ]:
true_effect = data_train['mu1'] - data_train['mu0']

In [ ]:
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

In [ ]:
means, stds = np.mean(influence_sample, axis=0), sem(influence_sample, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))